# Session 2 — FastAPI Routing

**Goals:**
- Understand routing fundamentals with real-world examples
- Build endpoints with path & query parameters and validation
- Learn common pitfalls and debugging techniques
- Practice with hands-on exercises and curl/httpx examples

This notebook is written for absolute beginners.

## 1) Real-world scenarios where routing is used

- E-commerce site: `/products`, `/products/{id}`, `/cart`, `/checkout`.
- Social app: `/users/{id}`, `/posts`, `/posts/{id}/comments`.
- Food delivery: `/restaurants`, `/menu/{restaurant_id}`, `/orders`.

Each click or screen load in these apps triggers one or more route calls to backend APIs.

## 2) Routing basics — method + path = handler

- **Method:** GET, POST, PUT, PATCH, DELETE
- **Path:** `/users/123` (may include path parameters)

When a request arrives, FastAPI finds the matching route and calls your handler function with parsed parameters.

In [ ]:
# Minimal app to demonstrate routing (save at app/main.py)
from fastapi import FastAPI

app = FastAPI(title='Routing - Minimal Demo')

@app.get('/hello')
def hello():
    return {'message': 'Hello from FastAPI'}

# Run: uvicorn app.main:app --reload --port 8000


## 3) APIRouter — modularize your routes (why it's useful)

- Group endpoints by feature
- Apply prefixes, tags, and shared dependencies
- Keep `main.py` small and readable

Typical structure:
```
app/
  routers/
    users.py
    items.py
  main.py
```

In [ ]:
# File: app/routers/users.py
from fastapi import APIRouter

router = APIRouter(prefix='/users', tags=['users'])

@router.get('/')
def list_users():
    return [{'id': 1, 'name': 'Alice'}]

@router.get('/{user_id}')
def get_user(user_id: int):
    return {'id': user_id, 'name': f'User {user_id}'}


## 4) Path parameters — required and type-checked

Path parameters are part of the URL and required. FastAPI converts and validates them using Python types.

In [ ]:
# File: app/routers/items.py (path params example)
from fastapi import APIRouter, Path

router = APIRouter(prefix='/items', tags=['items'])

@router.get('/{item_id}', summary='Get item by ID')
def get_item(item_id: int = Path(..., ge=1, description='Numeric ID of the item')):
    return {'item_id': item_id, 'name': f'Item {item_id}'}


### What happens if path param is wrong?
- If you call `/items/abc`, FastAPI will return 422 with a JSON error saying conversion failed.
- This is helpful for clients and reduces manual parsing code.

## 5) Query parameters — filters, pagination, options

Query params are optional unless you make them required. They are great for search, filters and pagination.

In [ ]:
# Query params example (app/routers/search.py)
from fastapi import APIRouter, Query

router = APIRouter(prefix='/search', tags=['search'])

@router.get('/items')
def search_items(q: str = Query(None, min_length=3, max_length=50), page: int = Query(1, ge=1), limit: int = Query(10, ge=1, le=100)):
    return {'q': q, 'page': page, 'limit': limit, 'results': []}


### Example URLs
- `/search/items?q=phone&page=2`  
- `/search/items?limit=5`  

Try these in your browser or Swagger.

## 6) Lists in query parameters

You can accept repeated query parameters: `/items?tag=red&tag=blue` which becomes a list in Python.

In [ ]:
# List query params example
from typing import List
from fastapi import APIRouter, Query

router = APIRouter(prefix='/filter', tags=['filter'])

@router.get('/items')
def filter_items(tag: List[str] = Query(None)):
    # tag will be None or a list like ['red', 'blue']
    return {'tags': tag}


## 7) Required query params (use Query(...))

If you want a non-path param to be required, use `...` inside Query/Body declaration.

In [ ]:
# Required query param example
from fastapi import APIRouter, Query

router = APIRouter(prefix='/required', tags=['required'])

@router.get('/check')
def check(q: str = Query(..., min_length=1)):
    return {'q': q}


## 8) Route matching order & pitfalls (declare specific first)

Routes are matched in the order FastAPI sees them. Specific routes (like `/me`) must come before parameterized ones (`/{username}`).

In [ ]:
# Order matters - example
from fastapi import APIRouter

router = APIRouter(prefix='/users', tags=['users'])

@router.get('/me')
def read_me():
    return {'user': 'current'}

@router.get('/{username}')
def read_user(username: str):
    return {'user': username}


## 9) Path validation with regex & special types

Use `Path(..., regex=...)` for custom patterns. You can also use `UUID` or `datetime` types for automatic parsing.

In [ ]:
# Regex and UUID examples
from uuid import UUID
from fastapi import APIRouter, Path

router = APIRouter(prefix='/resources', tags=['resources'])

@router.get('/by-uuid/{res_id}')
def get_by_uuid(res_id: UUID):
    return {'res_id': str(res_id)}

@router.get('/code/{code}')
def get_by_code(code: str = Path(..., regex='^[A-Z]{3}-\d{4}$')):
    return {'code': code}


## 10) Versioning strategies

Simple approach: prefix routers with `/v1`, `/v2`.
Alternative: header-based or content negotiation (advanced).

For beginners, use URL prefixing.

In [ ]:
# Example - include versioned routers (main.py)
from fastapi import FastAPI
# assume app.routers.items_v1 and items_v2 exist
# from app.routers import items_v1, items_v2

app = FastAPI(title='Versioning Example')

# app.include_router(items_v1.router, prefix='/v1')
# app.include_router(items_v2.router, prefix='/v2')

print('Include routers with prefix /v1 and /v2')

## 11) Dependencies & shared logic across routes (intro)

Use `Depends` to inject things like DB sessions, auth checks or common parameters. This keeps route handlers focused on business logic.

In [ ]:
# Simple Depends example
from fastapi import APIRouter, Depends, HTTPException

router = APIRouter(prefix='/protected', tags=['protected'])

def get_token(token: str = 'secret'):
    # very simple fake check for demo only
    if token != 'secret':
        raise HTTPException(status_code=401, detail='Unauthorized')
    return {'user': 'demo'}

@router.get('/me')
def me(user=Depends(get_token)):
    return {'user': user['user']}


## 12) Testing routes: curl and httpx examples

Use `curl` from terminal or `httpx` in Python to test endpoints. Swagger UI (`/docs`) is the easiest for beginners.

In [ ]:
# curl examples (run in terminal)
# curl 'http://127.0.0.1:8000/items/5'
# curl 'http://127.0.0.1:8000/search/items?q=phone&page=2'
print('Run the curl commands in your terminal; examples are commented out')

In [ ]:
# httpx quick example (save as call_test.py)
import httpx, asyncio

async def main():
    async with httpx.AsyncClient() as client:
        r = await client.get('http://127.0.0.1:8000/search/items?q=phone')
        print('status', r.status_code)
        print('json', r.json())

if __name__ == '__main__':
    asyncio.run(main())


## 13) Common mistakes & debugging tips

- Wrong module path when running uvicorn (use `uvicorn app.main:app`).
- Router not included in main app — endpoints won't register.
- Path type mismatch -> 422 errors.
- Overlapping routes capture unexpected paths.

**Debug tip:** Check `http://127.0.0.1:8000/docs` to see all registered routes and their expected parameters.

## 14) Exercises (step-by-step for freshers)

1. Create `app/routers/items.py` with path param example and include it in `app/main.py`.
2. Add a search endpoint that accepts `q`, `page`, `limit` and validate them.
3. Implement `books` CRUD router and test with curl and Swagger.
4. Add a protected route using a simple dependency `Depends`.

---

When you're done, run the server and check `/docs`. Want me to also add unit tests (pytest + httpx) for these routes?